# EPF Toolbox: forecasting models demo

Second notebook to generate day-ahead forecasts with `epftoolbox.models`.
- Mirrors the examples in `examples/` and the [models documentation](https://epftoolbox.readthedocs.io/en/latest/modules/models.html).
- Reuses data cached in `datasets/` from `01_load_data.ipynb`.
- Saves forecasts to `forecasts_local/` (created below).


In [1]:
from pathlib import Path

repo_root = Path.cwd().resolve()
if repo_root.name == 'Notebooks':
    repo_root = repo_root.parent

data_path = repo_root / 'datasets'
work_path = repo_root / 'forecasts_local'
work_path.mkdir(exist_ok=True)

print(f'Repo root: {repo_root}')
print(f'Data path: {data_path}')
print(f'Experiments path: {work_path}')


Repo root: /home/llinfeng/GitRepo/1_Projects/DianLi_电力/Benchmark1-epftoolbox
Data path: /home/llinfeng/GitRepo/1_Projects/DianLi_电力/Benchmark1-epftoolbox/datasets
Experiments path: /home/llinfeng/GitRepo/1_Projects/DianLi_电力/Benchmark1-epftoolbox/forecasts_local


# Verify GPU compute
In Temrinal, run: 
```bash
watch -n0.2 nvidia-smi  
```

In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices("GPU"))
with tf.device("/GPU:0"):
     a = tf.random.normal([2000, 2000])
     b = tf.random.normal([2000, 2000])
     _ = tf.matmul(a, b)  # warms up JIT

2025-12-01 15:40:06.936095: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-01 15:40:06.966093: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-01 15:40:06.966138: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-01 15:40:06.967063: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-01 15:40:06.973520: I tensorflow/core/platform/cpu_feature_guar

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2025-12-01 15:40:08.613743: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-12-01 15:40:08.613793: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-12-01 15:40:08.613800: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2022] Could not identify NUMA node of platform GPU id 0, defaulting to 0.  Your kernel may not have been built with NUMA support.
2025-12-01 15:40:08.613826: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-12-01 15:40:08.613855: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created

In [3]:
import tensorflow as tf, time

tf.debugging.set_log_device_placement(True)  # optional, see placements

with tf.device("/GPU:0"):
    a = tf.random.normal([8192, 8192])
    b = tf.random.normal([8192, 8192])
    for i in range(3):
        t0 = time.time()
        c = tf.matmul(a, b)
        _ = c.numpy()  # force materialization
        print(f"iter {i} took {time.time() - t0:.3f}s")

iter 0 took 1.078s
iter 1 took 0.878s
iter 2 took 0.866s


## LEAR daily recalibration (CPU extensive)

Quick run of the LEAR example (`epftoolbox.models.evaluate_lear_in_test_dataset`) to produce a forecast CSV in `forecasts_local/`.


In [ ]:
%%time
from epftoolbox.models import evaluate_lear_in_test_dataset

dataset = 'PJM'
years_test = 2
calibration_window = 364 * 3

forecast_lear = evaluate_lear_in_test_dataset(
    path_datasets_folder=str(data_path),
    path_recalibration_folder=str(work_path),
    dataset=dataset,
    years_test=years_test,
    calibration_window=calibration_window,
)

forecast_lear.head()


/home/llinfeng/GitRepo/1_Projects/DianLi_电力/Benchmark1-epftoolbox/.venv/lib/python3.11/site-packages/hyperopt/atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


## DNN daily recalibration (requires hyperparameters)

The DNN helpers mirror `examples/recalibrating_dnn_simplified.py`, but they need a trials file from `examples/optimizing_hyperparameters_dnn.py`.
The cell below checks for the expected file in `forecasts_local/` and only runs when it exists.


### Optional: create DNN hyperparameter trials

The DNN helper needs a trials file (hyperparameter search). If it is missing, run a small hyperopt sweep here to generate it. Increase `max_evals` for a proper search; the default keeps this demo quick.


In [ ]:
from epftoolbox.models import hyperparameter_optimizer

nlayers = 2
shuffle_train = 1
data_augmentation = 0
calibration_window = 4
years_test = 2
dataset = 'PJM'
experiment_id = 1
begin_test_date = None
end_test_date = None
max_evals = 10  # increase for a full search; small for a quick demo

trials_file_name = (
    f'DNN_hyperparameters_nl{nlayers}_dat{dataset}_YT{years_test}'
    f"{'_SF' if shuffle_train else ''}"
    f"{'_DA' if data_augmentation else ''}"
    f'_CW{calibration_window}_{experiment_id}'
)
trials_path = work_path / trials_file_name

if trials_path.exists():
    print('Trials file already exists, skipping hyperparameter search:')
    print(trials_path)
else:
    print('Running a small hyperparameter search to create trials file...')
    hyperparameter_optimizer(
        path_datasets_folder=str(data_path),
        path_hyperparameters_folder=str(work_path),
        new_hyperopt=True,
        max_evals=max_evals,
        nlayers=nlayers,
        dataset=dataset,
        years_test=years_test,
        calibration_window=calibration_window,
        shuffle_train=bool(shuffle_train),
        data_augmentation=bool(data_augmentation),
        experiment_id=experiment_id,
        begin_test_date=begin_test_date,
        end_test_date=end_test_date,
    )
    print('Saved trials to:', trials_path)


Running a small hyperparameter search to create trials file...


2025-12-01 15:28:40.724507: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-12-01 15:28:40.729920: W external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:225] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 12.0
2025-12-01 15:28:40.729951: W external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:228] Used ptxas at /home/llinfeng/GitRepo/1_Projects/DianLi_电力/Benchmark1-epftoolbox/.venv/lib/python3.11/site-packages/tensorflow/python/platform/../../../nvidia/cuda_nvcc/bin/ptxas
2025-12-01 15:28:40.730019: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-12-01 15:28:42.754334: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-



Tested 1/10 iterations.
Best MAE - Validation Dataset
  MAE: 4702943.6 | sMAPE: 41.72 %

Best MAE - Test Dataset
  MAE: 11.8 | sMAPE: 41.24 %


2025-12-01 15:28:59.693611: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-12-01 15:28:59.769174: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-12-01 15:28:59.769227: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-12-01 15:28:59.769686: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-12-01 15:28:59.771010: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-12-01



Tested 2/10 iterations.
Best MAE - Validation Dataset
  MAE: 7.3 | sMAPE: 17.13 %

Best MAE - Test Dataset
  MAE: 4.9 | sMAPE: 17.76 %


...



Tested 6/10 iterations.
Best MAE - Validation Dataset
  MAE: 6.1 | sMAPE: 13.97 %

Best MAE - Test Dataset
  MAE: 3.9 | sMAPE: 14.85 %




Tested 7/10 iterations.
Best MAE - Validation Dataset
  MAE: 6.1 | sMAPE: 13.97 %

Best MAE - Test Dataset
  MAE: 3.9 | sMAPE: 14.85 %




Tested 8/10 iterations.
Best MAE - Validation Dataset
  MAE: 6.1 | sMAPE: 13.97 %

Best MAE - Test Dataset
  MAE: 3.9 | sMAPE: 14.85 %




Tested 9/10 iterations.
Best MAE - Validation Dataset
  MAE: 6.1 | sMAPE: 13.97 %

Best MAE - Test Dataset
  MAE: 3.9 | sMAPE: 14.85 %


Saved trials to: /home/llinfeng/GitRepo/1_Projects/DianLi_电力/Benchmark1-epftoolbox/forecasts_local/DNN_hyperparameters_nl2_datPJM_YT2_SF_CW4_1


In [ ]:
%%time
from epftoolbox.models import evaluate_dnn_in_test_dataset

nlayers = 2
shuffle_train = 1
data_augmentation = 0
calibration_window = 4
years_test = 2
dataset = 'PJM'
experiment_id = 1
begin_test_date = None
end_test_date = None

trials_file_name = (
    f'DNN_hyperparameters_nl{nlayers}_dat{dataset}_YT{years_test}'
    f"{'_SF' if shuffle_train else ''}"
    f"{'_DA' if data_augmentation else ''}"
    f'_CW{calibration_window}_{experiment_id}'
)
trials_path = work_path / trials_file_name

if trials_path.exists():
    forecast_dnn = evaluate_dnn_in_test_dataset(
        experiment_id=experiment_id,
        path_hyperparameter_folder=str(work_path),
        path_datasets_folder=str(data_path),
        path_recalibration_folder=str(work_path),
        nlayers=nlayers,
        dataset=dataset,
        years_test=years_test,
        shuffle_train=bool(shuffle_train),
        data_augmentation=bool(data_augmentation),
        calibration_window=calibration_window,
        new_recalibration=True,
        begin_test_date=begin_test_date,
        end_test_date=end_test_date,
    )
    display(forecast_dnn.head())
else:
    print('Skipping DNN run – hyperparameter trials file not found:')
    print(trials_path)
    print('Run examples/optimizing_hyperparameters_dnn.py to generate it.')

2016-12-27 - sMAPE: 3.83%  |  MAE: 0.931


2016-12-28 - sMAPE: 7.29%  |  MAE: 1.889


2016-12-29 - sMAPE: 8.24%  |  MAE: 2.154


2016-12-30 - sMAPE: 7.17%  |  MAE: 1.916


2016-12-31 - sMAPE: 6.93%  |  MAE: 1.851


...

2017-01-02 - sMAPE: 6.62%  |  MAE: 1.784


2017-01-03 - sMAPE: 7.32%  |  MAE: 1.975


2017-01-04 - sMAPE: 7.23%  |  MAE: 1.999


2017-01-05 - sMAPE: 8.28%  |  MAE: 2.469
2017-01-06 - sMAPE: 8.41%  |  MAE: 2.569


KeyboardInterrupt: 